In [1]:
import json
import torch
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import datasets
from peft import get_peft_model
from peft import LoraConfig
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer
from transformers import TrainingArguments

In [2]:
# 训练集共1949972条，只取前1000条
# with open("medical_zh/train_zh_0.jsonl", "r") as f:
#     lst = [json.loads(next(f)) for _ in range(1000)]
# lst[0]

In [3]:
# with open("train_lora.json", "w") as f:
#     json.dump(lst, f, ensure_ascii=False)  # 有汉字，加ensure_ascii=False

In [4]:
# 加载Dataset。原文经过DataFrame中转，比较麻烦
ds = datasets.load_dataset("json", data_files="train_lora.json", split="train")  # 不加split会返回DatasetDict
ds[0]

{'input': '',
 'output': '初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。',
 'instruction': '血热的临床表现是什么?'}

In [5]:
CKPT_PATH = "Qwen-1_8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(CKPT_PATH, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eod_id
tokenizer

QWenTokenizer(name_or_path='Qwen-1_8B-Chat', vocab_size=151851, model_max_length=8192, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [6]:
def process_func(example):
    MAX_LENGTH = 384

    instruction = tokenizer(
        "<|im_start|>system\n" +
        "你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>\n" +
        "<|im_start|>user\n" +
        example["instruction"] + example["input"] + "<|im_end|>\n"
    )
    response = tokenizer(
        "<|im_start|>assistant\n" +
        example["output"] + "<|im_end|>\n"
    )
    
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eod_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eod_id]

    # 截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
# 类似DataFrame的用法，并行处理数据
train_dataset = ds.map(process_func, remove_columns=ds.column_names)  # 去掉原始的列
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [8]:
tokenizer.decode(train_dataset[1]['input_ids'])

'<|im_start|>system\n你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>\n<|im_start|>user\n帕金森叠加综合征的辅助治疗有些什么？<|im_end|>\n<|im_start|>assistant\n综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗<|im_end|>\n<|endoftext|>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, train_dataset[1]["labels"])))

'<|im_start|>assistant\n综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗<|im_end|>\n<|endoftext|>'

In [10]:
model = AutoModelForCausalLM.from_pretrained(CKPT_PATH, trust_remote_code=True,
                                             bf16=True,
                                             torch_dtype=torch.bfloat16,
                                             use_flash_attn = False,
                                             # load_in_8bit=True
                                            )
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-23): 24 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear(in_features=2048, out_features=5504, bias=False)
          (w2): Linear(in_features=2048, out_features=5504, bias=False)
          (c_proj): Linear(in_features=5504, out_features=2048, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=151936, bias=False)
)

In [11]:
model.dtype

torch.bfloat16

In [12]:
# 使用gradient_checkpointing+peft需要加这句补丁
model.enable_input_require_grads()

In [13]:
config = LoraConfig(
    # task_type=TaskType.CAUSAL_LM, # 可以不写
    target_modules=["c_attn", "c_proj", "w1", "w2"],  # 必须指定，Qwen官方的微调脚本也是写这几个
    # 原文说这三个参数比较通用，先留着
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
# config

In [14]:
model = get_peft_model(model, config)
# model

In [15]:
model.print_trainable_parameters()

trainable params: 6,709,248 || all params: 1,843,537,920 || trainable%: 0.3639


In [16]:
# 定义训练配置
args = TrainingArguments(
    output_dir=f"./output/{CKPT_PATH}_flash_attn_v2",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    logging_steps=10,
    num_train_epochs=1,
    gradient_checkpointing=True,  # 开启激活重计算，时间换空间
    # save_steps=50,
    learning_rate=1e-4
)

In [17]:
# 定义trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    # 用pad填充batch的三个字段
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)  # 原文设置padding=True，是默认值，省略掉
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
# 训练前看看效果
# response, history = model.eval().chat(tokenizer, "帕金森叠加综合征的辅助治疗有些什么?", history=None,
#                                       system="你是一个医学助手，需要回答用户关于医学的问题：")
# response, history

In [19]:
result = trainer.train()
result

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/pai/envs/qwen/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.442200
20,2.203100
30,2.151600
40,2.115600
50,2.085200
60,2.092200


TrainOutput(global_step=63, training_loss=2.1889880952380953, metrics={'train_runtime': 115.8858, 'train_samples_per_second': 8.629, 'train_steps_per_second': 0.544, 'total_flos': 3475127730241536.0, 'train_loss': 2.1889880952380953, 'epoch': 1.0})

In [20]:
# 训练后看看效果（用回训练集测试效果当然好）
# response, history = model.eval().chat(tokenizer, "帕金森叠加综合征的辅助治疗有些什么?", history=None,
#                                       system="你是一个医学助手，需要回答用户关于医学的问题：")
# response, history